# TODO: change everything (placeholder file)

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from PIL import Image
import imageio, skimage

from collections import Counter

In [ ]:
def show_book_figure(figure_id: str):
    #print('figure_id:', figure_id)
    display(bookset_captions_df[bookset_captions_df['figure_id'] == figure_id])

    for i, row in bookset_captions_df[bookset_captions_df.figure_id == figure_id].sort_values(by='letter').iterrows():
        letter = row[1]
        img_uuid = row[3]
        print('fugire_id:', figure_id)
        print(letter, ':', img_uuid)

        img = imageio.imread(f"{book_set_dir}/images/{img_uuid}.png")
        plt.imshow(img)
        plt.show()

In [ ]:
def show_pubmed_figure(caption: str):
    print('caption:', caption)
    display(pubmed_captions_df[pubmed_captions_df['caption'] == caption])

    for i, row in pubmed_captions_df[pubmed_captions_df.caption == caption].sort_values(by='uuid').iterrows():
        #letter = row[1]
        img_uuid = row[1]
        #print(letter, ':', img_uuid)

        img = imageio.imread(f"{pubmed_set_dir}/images/{img_uuid}.jpg")
        plt.imshow(img)
        plt.show()

In [ ]:
def counter_to_dataframe(c: Counter):

    bag_sizes_to_freqs = {
        'bag_size': [],
        'freq': []
    }

    for bag_size, freq in c.items():
        bag_sizes_to_freqs['bag_size'].append(bag_size)
        bag_sizes_to_freqs['freq'].append(freq)

    bag_sizes_to_freqs_df = pd.DataFrame(bag_sizes_to_freqs).sort_values(by='bag_size')
    bag_sizes_to_freqs_df = bag_sizes_to_freqs_df.set_index('bag_size')
    display(bag_sizes_to_freqs_df)

In [ ]:
os.listdir('../datasets/ARCH')

## Books set

In [ ]:
book_set_dir = '../datasets/ARCH/books_set'
os.listdir(book_set_dir)

### Readme

In [ ]:
!cat ../datasets/ARCH/books_set/README.md

### Images

In [ ]:
len(os.listdir(f'{book_set_dir}/images'))

In [ ]:
bookset_image_uuids = [f_name.split('.')[0] for f_name in os.listdir(f'{book_set_dir}/images')]
assert len(bookset_image_uuids) == len(set(bookset_image_uuids))

len(bookset_image_uuids)

### Captions

In [ ]:
with open(f'{book_set_dir}/captions.json', 'r') as f:
    bookset_captions = json.load(f)

In [ ]:
bookset_captions_df = pd.DataFrame(bookset_captions).T
bookset_captions_df

In [ ]:
counter_to_dataframe(Counter(bookset_captions_df.caption.value_counts().values))

In [ ]:
bookset_captions_all_images_present_df = bookset_captions_df[bookset_captions_df.uuid.isin(bookset_image_uuids)]
bookset_captions_all_images_present_df

In [ ]:
counter_to_dataframe(Counter(bookset_captions_all_images_present_df.caption.value_counts().values))

### Number of images per bag

#### Missing Images present

In [ ]:
Counter(Counter(bookset_captions_df.figure_id).values())

In [ ]:
counter_to_dataframe(Counter(Counter(bookset_captions_df.figure_id).values()))

In [ ]:
bookset_captions_df.nunique()

#### w/o missing images

In [ ]:
# use only the information about the images present in the ..datasets/ARCH/books_set/images/
bookset_captions_all_images_present_df = bookset_captions_df[bookset_captions_df.uuid.isin(bookset_image_uuids)]
counter_to_dataframe(Counter(Counter(bookset_captions_all_images_present_df.figure_id).values()))

## PubMed Set

In [ ]:
pubmed_set_dir = '../datasets/ARCH/pubmed_set'

In [ ]:
os.listdir(pubmed_set_dir)

### Readme

In [ ]:
!cat ../datasets/ARCH/pubmed_set/README.md

### Captions

In [ ]:
with open(f'{pubmed_set_dir}/captions.json', 'r') as f:
    pubmed_captions = json.load(f)

pubmed_captions

Indices appear to be integers from 0 to 3308.

In [ ]:
# Check for duplicates (slightly pointless since keys have to be unique anyway)
assert len(pubmed_captions.keys()) == len(set(pubmed_captions.keys()))

# check for the indices to be without gaps ['0', '1', '2', ..., '3308']
assert sorted(list(pubmed_captions.keys())) == sorted([str(i) for i in range(len(pubmed_captions.keys()))])

A simple check confirms that everything is in order.

In [ ]:
pubmed_captions_df = pd.DataFrame(pubmed_captions).T
pubmed_captions_df

In [ ]:
pubmed_captions_df.nunique()

In [ ]:
captions_with_multiple_occurrences = [(caption, count) \
                                      for caption, count in Counter(pubmed_captions_df.caption).items() \
                                      if count > 1]
captions_with_multiple_occurrences

In [ ]:
# counting extra uuids (substract 1 since 1 uuid per caption should be there by default)
sum([num-1 for caption, num in captions_with_multiple_occurrences])

In [ ]:
len(captions_with_multiple_occurrences)

In [ ]:
Counter(pubmed_captions_df.caption)

In [ ]:
caption_of_interest = 'Low-grade peritoneal serous carcinoma. Monomorphic cells associated with psammoma bodies are seen. Hematoxylin and eosin stain, original magnification: x100.'
pubmed_captions_df[pubmed_captions_df.caption == caption_of_interest]

### Images

In [ ]:
len(os.listdir(f'{pubmed_set_dir}/images'))

In [ ]:
pubmed_image_ids = [f_name.split('.')[0] for f_name in os.listdir(f'{pubmed_set_dir}/images')]
assert len(pubmed_image_ids) == len(set(pubmed_image_ids))

len(pubmed_image_ids)

In [ ]:
set(pubmed_image_ids) == set(pubmed_captions_df.uuid)

All UUIDs in the `captions.json` coincide with the UUIDs of the images in the folder. No Images missing.

In [ ]:
for caption_of_interest, count in captions_with_multiple_occurrences:
    show_pubmed_figure(caption_of_interest)

Captions are not split into different images. There are no "A", "B", "C" parts in a caption. There are also no "A", "B", "C" labels on images.

In [ ]:
Counter(Counter(pubmed_captions_df.caption).values())

In [ ]:
counter_to_dataframe(Counter(Counter(pubmed_captions_df.caption).values()))

In [ ]:
pubmed_captions_df.nunique()

In [ ]:
pubmed_captions_df['uuid'].nunique() - pubmed_captions_df['caption'].nunique()